# Script to align human typicality ratings with the model scores

In [1]:
import os
import pandas as pd
import numpy as np

print(os.getcwd())

/Users/carinakauf/repos/lm-event-knowledge/analyses


## Read in relevant files and filter out duplicate sentences

In [2]:
ratings_file = "/Users/carinakauf/repos/lm-event-knowledge/human_ratings/3_DTFIT/TypicalityRatings_Triples.csv"
sentences_file = "/Users/carinakauf/repos/lm-event-knowledge/model_scores/DTFit_scores_new_format/dtfit_vassallo_deps.prob_sent_gpt2-xl.txt"

In [3]:
ratings_df = pd.read_csv(ratings_file)
ratings_df

,SUBJECT,VERB,OBJECT,typicality,mean_rat,std_rat
0,woman,paint,toenail,T,6.10,1.51
1,reporter,hold,camera,T,4.50,1.57
2,programmer,use,computer,T,7.00,0.00
3,mason,mix,cement,T,6.65,0.65
4,bodybuilder,lift,weight,T,6.45,1.66
...,...,...,...,...,...,...
801,pupil,draw,turbine,AT,2.95,1.63
802,veterinarian,examine,epigraph,AT,1.55,0.80
803,child,write,diagnosis,AT,1.05,0.22
804,archeologist,examine,dog,AT,1.60,0.92


In [4]:
#summarize duplicates
grouped_df = ratings_df.groupby(["SUBJECT","VERB", "OBJECT", "typicality"])["mean_rat"].mean()
grouped_df = grouped_df.reset_index()
grouped_df

,SUBJECT,VERB,OBJECT,typicality,mean_rat
0,actor,win,award,T,5.80
1,actor,win,battle,AT,2.60
2,anchorman,tell,news,T,6.75
3,anchorman,tell,parable,AT,3.00
4,animal,find,food,T,5.85
...,...,...,...,...,...
799,woman,carry,stone,AT,3.10
800,woman,open,bag,T,6.45
801,woman,open,manhole,AT,2.40
802,woman,paint,sign,AT,3.55


In [16]:
triplet = list(map(list, zip(list(grouped_df["SUBJECT"]),list(grouped_df["VERB"]),list(grouped_df["OBJECT"]))))
print(len(list(np.unique(triplet))))

cnt_dict = {}
for elm in triplet:
    if "+".join(elm) not in cnt_dict:
        cnt_dict["+".join(elm)] = 1
    else:
        cnt_dict["+".join(elm)] += 1
        
for key, value in cnt_dict.items():
    if value > 1:
        print(key, value)

589


In [17]:
triplet_rtg = list(map(list, zip(triplet, list(grouped_df["typicality"]), list(grouped_df["mean_rat"]))))
print(len(triplet_rtg))
triplet_rtg[:5]

804


[[['actor', 'win', 'award'], 'T', 5.8],
 [['actor', 'win', 'battle'], 'AT', 2.6],
 [['anchorman', 'tell', 'news'], 'T', 6.75],
 [['anchorman', 'tell', 'parable'], 'AT', 3.0],
 [['animal', 'find', 'food'], 'T', 5.85]]

### change some words which are different in the DTFit columns and the sentence
Maybe `collect` should be excluded?

In [7]:
#manually change some based on DTFit data:
# we could also exclude these sentences!! (at least the last two)
curated_triplet_rtg = []
for elm in triplet_rtg:
    if "grocery"==elm[0][2]:
        elm[0][2] = "groceries"
        curated_triplet_rtg.append(elm)
    elif "skateboard"==elm[0][2]:
        elm[0][2] = "skates"
        curated_triplet_rtg.append(elm)
    elif (elm[0][0] == "binman") and (elm[0][1] == "pick"):
        elm[0][1] = "collect"
        curated_triplet_rtg.append(elm)
    else:
        curated_triplet_rtg.append(elm)

In [8]:
sentences_df = pd.read_csv(sentences_file, sep="\t")
del sentences_df["0"]
sentences_df.columns = ["sentence", "score"]
sentences_df.head()

,sentence,score
0,The actor won the award.,2.506995e-18
1,The anchorman told the parable.,5.052057e-25
2,The anchorman told the news.,1.460487e-20
3,The animal found the map.,4.536541e-23
4,The animal found the food.,2.643247e-22


In [9]:
#check for sentence duplicates
sentences = []
sentences_dict = {}
for ind, row in sentences_df.iterrows():
    if row["sentence"] not in sentences_dict:
        sentences_dict[row["sentence"]] = [row["score"]]
        sentences.append(row["sentence"])
    else:
        sentences_dict[row["sentence"]].append(row["score"])
        print(row["sentence"], ind)

print("*"*30)
for key, value in sentences_dict.items():
    if len(value) > 1:
        print(key, value)
        assert value[0] == value[1]
        print(">> asserted")
        
#deleted two duplicate sentences
print(len(sentences_df))
print(len(sentences))

The soldier threw the bomb. 644
The student wore the glasses. 692
******************************
The soldier threw the bomb. [1.4658519333840488e-19, 1.4658519333840488e-19]
>> asserted
The student wore the glasses. [5.376707908924492e-21, 5.376707908924492e-21]
>> asserted
797
795


In [10]:
irregular_verbs = pd.read_csv("irregularVerbs_WithPresent.csv")
irregular_verbs

present_verbs = list(irregular_verbs["present"])
past_verbs = list(irregular_verbs["simple_past"])

irregular_verbs

,present,simple_past,past_participle
0,are,were,been
1,is,was,been
2,beat,beat,beaten
3,become,became,become
4,begin,began,begun
...,...,...,...
147,wet,wet,wet
148,win,won,won
149,wind,wound,wound
150,wring,wrung,wrung


## Align triplets & scores with sentences.
Hacky match of verbs which are not past-tensed in the human-ratings file

In [11]:
typicality = []
human_ratings = []
sorted_triplets = []

for sent in sentences:
    words = sent.split()
    words = [elm.rstrip(".") for elm in words]
    cnt = 0
    matched_triplets = []
    for triplet in curated_triplet_rtg:
        subj_obj = [triplet[0][0], triplet[0][2]]
        if all(substring in words for substring in subj_obj):
            verb = triplet[0][1]
            try:
                idx = present_verbs.index(verb)
                past_verb = past_verbs[idx]
            except:
                if verb == "enjoy":
                    past_verb = "enjoyed"
                elif verb == "play":
                    past_verb = "played"
                elif verb == "pick":
                    past_verb = "picked"
                elif verb == "spray":
                    past_verb = "sprayed"
                elif verb == "help":
                    past_verb = "helped"
                elif verb == "grab":
                    past_verb = "grabbed"
                elif verb[-1] == "y":
                    past_verb = verb[:-1] + "ied"
                elif verb[-1] == "e":
                    past_verb = verb + "d"
                elif verb[-1] == "p":
                    past_verb = verb + "ped"
                else:
                    past_verb = verb + "ed"
            if words[2] == past_verb:
                #print(sent, triplet)
                sorted_triplets.append(triplet[0])
                typicality.append(triplet[1])
                human_ratings.append(triplet[2])
                matched_triplets.append(triplet)
    if len(matched_triplets) > 1:
        print(sent, matched_triplets)
                
    if len(matched_triplets) == 0:
        print(f"No matching triplet found for sentence: {sent}")

## Save aligned ratings to dataframe

In [12]:
print(len(sentences))
print(len(typicality))
print(len(human_ratings))
print(len(sorted_triplets))

795
795
795
795


In [13]:
human_ratings_df = pd.DataFrame({
    "sentence": sentences,
    "typicality": typicality,
    "rating": human_ratings,
    "subject/object" : sorted_triplets
})
human_ratings_df

,sentence,typicality,rating,subject/object
0,The actor won the award.,T,5.80,"[actor, win, award]"
1,The anchorman told the parable.,AT,3.00,"[anchorman, tell, parable]"
2,The anchorman told the news.,T,6.75,"[anchorman, tell, news]"
3,The animal found the map.,AT,2.00,"[animal, find, map]"
4,The animal found the food.,T,5.85,"[animal, find, food]"
...,...,...,...,...
790,The woman carried the bag.,T,6.25,"[woman, carry, bag]"
791,The woman opened the manhole.,AT,2.40,"[woman, open, manhole]"
792,The woman opened the bag.,T,6.45,"[woman, open, bag]"
793,The woman painted the sign.,AT,3.55,"[woman, paint, sign]"


In [ ]:
savedir = "/Users/carinakauf/repos/lm-event-knowledge/human_ratings/3_DTFIT/"
filename = "curated_human_ratings.csv"
human_ratings_df.to_csv(os.path.join(savedir,filename))

In [18]:
del human_ratings_df["subject/object"]

savedir = "/Users/carinakauf/repos/lm-event-knowledge/human_ratings/3_DTFIT/"
filename = "newformat_curated_human_ratings.csv"
human_ratings_df.to_csv(os.path.join(savedir,filename))

In [22]:
filename = "newformat_curated_human_ratings.txt"
human_ratings_df.to_csv(os.path.join(savedir,filename), sep="\t",header=False)